# Import the required libraries

In [1]:
import pandas as pd
import numpy as np

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spaCy
import spacy
import nltk 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#vis 
import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

C:\ProgramData\Anaconda3\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


# Import data

In [2]:
imdb_data = pd.read_csv("imdb_labelled.txt",  names=['review','label'], delimiter=".\t")
imdb_data

C:\Users\Quynh Pham\AppData\Local\Temp\ipykernel_16424\3627405509.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  imdb_data = pd.read_csv("imdb_labelled.txt",  names=['review','label'], delimiter=".\t")


,review,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
995,I just got bored watching Jessice Lange take h...,0
996,"Unfortunately, any virtue in this film's produ...",0
997,"In a word, it is embarrassing.",0
998,Exceptionally bad!,0


In [3]:
review = imdb_data.review.tolist()
review[:10]

['A very, very, very slow-moving, aimless movie about a distressed, drifting young man. ',
 'Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out. ',
 'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent. ',
 'Very little music or anything to speak of. ',
 'The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head. ',
 "The rest of the movie lacks art, charm, meaning... If it's about emptiness, it works I guess because it's empty. ",
 'Wasted two hours. ',
 'Saw the movie today and thought it was a good effort, good messages for kids. ',
 'A bit predictable. ',
 'Loved the casting of Jimmy Buffet as the science teacher. ']

# Prepare data

In [4]:
# create a spacy object, disable parser and ner for the script to run a bit faster
nlp = spacy.load('en_core_web_sm', disable=['parser','ner']) 

#get the list of stop words
stopwords = stopwords.words('english')
stopwords.append('movie')
stopwords.append('film')
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
# function to preprocess the review
def preprocess(texts, allowed_postags=['NOUN','VERB','ADJ','ADV']):    
    lemmatizer = WordNetLemmatizer()
    filter_sentence = []
    temp = []
    words = gensim.utils.simple_preprocess(str(texts), deacc=True) # gensim - lowercase, tokenize, de-accents
    words = [w for w in words if not w in stopwords] # stopwords removal
    temp = ' '.join(x for x in words) # combine the tokens into a sentence again
    doc = nlp(temp)
    for word in doc: # go through each words in that clean sentence
        if word.pos_ in allowed_postags: # remove words that are not verb or noun
            filter_sentence.append(word.lemma_) # lemmatization
    return filter_sentence

In [6]:
# build the dictionary
corpus = [preprocess(line) for line in review]
corpus[:3]

[['slow', 'move', 'aimless', 'distressed', 'drift', 'young', 'man'],
 ['sure', 'lose', 'flat', 'character', 'audience', 'nearly', 'half', 'walk'],
 ['attempt',
  'artiness',
  'black',
  'white',
  'clever',
  'camera',
  'angle',
  'disappoint',
  'become',
  'even',
  'ridiculous',
  'act',
  'poor',
  'plot',
  'line',
  'almost',
  'existent']]

In [7]:
# build the dicationary with gensim
dictionary = corpora.Dictionary(corpus)
len(dictionary)

2122

In [8]:
# convert corpus into bag-of-words format
bow = [dictionary.doc2bow(line) for line in corpus]
print(bow[0][0:20])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]


# Build model with 2 topics

In [9]:
lda_model_1 = gensim.models.LdaModel(corpus=bow, id2word=dictionary, num_topics=2, passes=8, alpha="auto")

In [10]:
lda_model_1.show_topics(formatted=True, num_topics=2, num_words=20)

[(0,
  '0.014*"bad" + 0.012*"watch" + 0.011*"really" + 0.009*"good" + 0.007*"get" + 0.007*"character" + 0.006*"way" + 0.006*"story" + 0.006*"make" + 0.006*"much" + 0.005*"thing" + 0.005*"give" + 0.004*"plot" + 0.004*"funny" + 0.004*"time" + 0.004*"cast" + 0.004*"performance" + 0.004*"still" + 0.004*"never" + 0.003*"feel"'),
 (1,
  '0.016*"see" + 0.010*"good" + 0.010*"well" + 0.009*"act" + 0.009*"make" + 0.009*"bad" + 0.007*"great" + 0.007*"time" + 0.007*"character" + 0.007*"think" + 0.007*"love" + 0.006*"even" + 0.006*"go" + 0.006*"script" + 0.005*"ever" + 0.005*"scene" + 0.005*"movie" + 0.005*"show" + 0.005*"look" + 0.005*"also"')]

In [11]:
# Visualize model with 2 topics
pyLDAvis.enable_notebook() #allow us to see in jupyter notebook
vis1 = pyLDAvis.gensim_models.prepare(lda_model_1, bow, dictionary)
vis1

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.076628  0.0       1        1  58.271937
0     -0.076628  0.0       2        1  41.728063, topic_info=       Term       Freq      Total Category  logprob  loglift
389   watch  33.000000  33.000000  Default  30.0000  30.0000
58      see  60.000000  60.000000  Default  29.0000  29.0000
300  really  29.000000  29.000000  Default  28.0000  28.0000
15      act  34.000000  34.000000  Default  27.0000  27.0000
445     way  16.000000  16.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
52     work   7.623526  20.873523   Topic2  -5.8077  -0.1332
172    play   7.540936  20.107190   Topic2  -5.8186  -0.1067
168     say   7.247555  14.711741   Topic2  -5.8583   0.1660
274   actor   7.362670  21.677459   Topic2  -5.8425  -0.2059
41    scene   7.347084  25.556064   Topic2  -5.8446  -0.3726

[150 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
288       1  0.908798  absolutely
15        1  0.950624         act
15        2  0.057614         act
718       1  0.866917      action
718       2  0.144486      action
...     ...       ...         ...
312       2  0.093095       worth
244       1  0.726768       write
244       2  0.302820       write
342       1  0.910215        year
342       2  0.065015        year

[184 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

# Build model with 5 topics

In [12]:
lda_model_2 = gensim.models.LdaModel(corpus=bow, id2word=dictionary, num_topics=5, passes=8, alpha="auto")

In [13]:
lda_model_2.show_topics(formatted=True, num_topics=5, num_words=20)

[(0,
  '0.017*"character" + 0.016*"good" + 0.010*"act" + 0.009*"time" + 0.008*"go" + 0.008*"make" + 0.008*"see" + 0.007*"plot" + 0.007*"watch" + 0.007*"bad" + 0.007*"end" + 0.006*"also" + 0.005*"still" + 0.005*"kid" + 0.005*"know" + 0.005*"scene" + 0.005*"great" + 0.005*"well" + 0.005*"write" + 0.004*"take"'),
 (1,
  '0.012*"suck" + 0.012*"well" + 0.011*"make" + 0.008*"look" + 0.007*"character" + 0.007*"beautiful" + 0.006*"time" + 0.006*"good" + 0.006*"much" + 0.006*"real" + 0.006*"enjoy" + 0.005*"great" + 0.005*"pretty" + 0.005*"like" + 0.005*"get" + 0.005*"work" + 0.005*"create" + 0.004*"totally" + 0.004*"plot" + 0.004*"long"'),
 (2,
  '0.020*"see" + 0.014*"bad" + 0.011*"think" + 0.010*"recommend" + 0.009*"make" + 0.009*"scene" + 0.009*"story" + 0.008*"movie" + 0.008*"ever" + 0.008*"excellent" + 0.006*"well" + 0.006*"awful" + 0.005*"film" + 0.005*"little" + 0.005*"definitely" + 0.005*"worth" + 0.005*"act" + 0.005*"art" + 0.005*"highly" + 0.005*"many"'),
 (3,
  '0.028*"bad" + 0.014*"s

In [14]:
# Visualize model with 5 topics
vis2 = pyLDAvis.gensim_models.prepare(lda_model_2, bow, dictionary)
vis2

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.039523  0.069145       1        1  21.706067
3     -0.075763  0.065197       2        1  21.357762
2     -0.093412 -0.073163       3        1  21.045280
4      0.058723  0.023628       4        1  20.715153
1      0.070929 -0.084806       5        1  15.175738, topic_info=          Term       Freq      Total Category  logprob  loglift
215        bad  69.000000  69.000000  Default  30.0000  30.0000
530       suck  11.000000  11.000000  Default  29.0000  29.0000
58         see  59.000000  59.000000  Default  28.0000  28.0000
59       think  24.000000  24.000000  Default  27.0000  27.0000
8    character  43.000000  43.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
192       also   3.458581  19.260174   Topic5  -5.5866   0.1683
87        ever   3.474111  22.233418   Topic5  -5.5821   0.0292
743        use   2.905151  13.268379   Topic5  -5.7610   0.3666
445        way   2.889028  17.023484   Topic5  -5.7665   0.1118
213      story   2.902553  24.533532   Topic5  -5.7619  -0.2490

[353 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
15        1  0.386894   act
15        2  0.208328   act
15        3  0.178567   act
15        4  0.119044   act
15        5  0.089283   act
...     ...       ...   ...
342       1  0.134808  year
342       2  0.404423  year
342       3  0.269616  year
342       4  0.067404  year
342       5  0.134808  year

[563 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 5, 2])

# Build model with 10 topics

In [15]:
lda_model_3 = gensim.models.LdaModel(corpus=bow, id2word=dictionary, num_topics=10, passes=8, alpha="auto")

In [16]:
lda_model_3.show_topics(formatted=True, num_topics=10, num_words=20)

[(0,
  '0.013*"well" + 0.013*"time" + 0.011*"really" + 0.011*"play" + 0.011*"waste" + 0.010*"line" + 0.010*"character" + 0.010*"good" + 0.009*"also" + 0.009*"cast" + 0.008*"plot" + 0.007*"pretty" + 0.007*"act" + 0.006*"see" + 0.006*"scene" + 0.006*"like" + 0.006*"performance" + 0.006*"story" + 0.006*"know" + 0.006*"write"'),
 (1,
  '0.050*"bad" + 0.026*"see" + 0.015*"ever" + 0.015*"make" + 0.012*"good" + 0.011*"scene" + 0.010*"way" + 0.009*"think" + 0.007*"script" + 0.007*"work" + 0.007*"movie" + 0.007*"go" + 0.007*"well" + 0.005*"terrible" + 0.005*"cast" + 0.005*"really" + 0.005*"song" + 0.005*"old" + 0.005*"like" + 0.005*"much"'),
 (2,
  '0.021*"see" + 0.017*"character" + 0.013*"funny" + 0.013*"really" + 0.012*"love" + 0.010*"script" + 0.010*"wonderful" + 0.009*"film" + 0.009*"actor" + 0.009*"great" + 0.009*"lot" + 0.009*"avoid" + 0.008*"even" + 0.006*"truly" + 0.006*"look" + 0.006*"screen" + 0.005*"make" + 0.005*"year" + 0.005*"comedy" + 0.005*"story"'),
 (3,
  '0.017*"great" + 0.01

In [17]:
# Visualize model with 10 topics
vis3 = pyLDAvis.gensim_models.prepare(lda_model_3, bow, dictionary)
vis3

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.040674  0.119324       1        1  14.451982
6     -0.059192 -0.010581       2        1  12.274415
1      0.115745 -0.030216       3        1  11.232752
4      0.123112  0.024149       4        1  11.049307
2      0.025491  0.092630       5        1  10.042648
8      0.032975 -0.072272       6        1   9.953588
3     -0.015129 -0.095390       7        1   9.034246
5     -0.052462  0.015769       8        1   8.308152
9     -0.079802 -0.066711       9        1   7.258190
7     -0.050062  0.023297      10        1   6.394720, topic_info=          Term       Freq      Total Category  logprob  loglift
215        bad  69.000000  69.000000  Default  30.0000  30.0000
389      watch  36.000000  36.000000  Default  29.0000  29.0000
530       suck  13.000000  13.000000  Default  28.0000  28.0000
25        even  30.000000  30.000000  Default  27.0000  27.0000
58         see  58.000000  58.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
118     minute   1.386506   9.634945  Topic10  -5.6364   0.8111
475        get   1.383991  23.314736  Topic10  -5.6383  -0.0744
454       fact   1.383676   6.681625  Topic10  -5.6385   1.1751
864  wonderful   1.382971  13.884642  Topic10  -5.6390   0.4431
202       tell   1.382744   5.883760  Topic10  -5.6392   1.3016

[716 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1074      4  0.671201      abroad
288       3  0.278324  absolutely
288       5  0.139162  absolutely
288       8  0.278324  absolutely
288      10  0.139162  absolutely
...     ...       ...         ...
342       6  0.067932        year
342       7  0.135864        year
342       8  0.067932        year
342      10  0.135864        year
1094      6  0.675656        yelp

[1401 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 2, 5, 3, 9, 4, 6, 10, 8])